## The pure BERT model for token classification

In [2]:
! python -V

Python 3.7.12


In [3]:
! pip install seqeval==0.0.12 #BERT-large-cased

In [4]:
! pip install pytorch-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 55.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 79.1 MB/s eta 0:00:0000:0100:01
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=b0f0273a536f3738798d6da317edf5061a815be8bd526d6a8824623e937ae1b1
  Stored in directory: /root/.cache/pip/wheels/5b/e0/77/05245143a5b31f65af6a21f7afd3219e9fa4896f918af45677
Successfully built sacremoses
  Attempting uninstall: botocore
    Found existing installation: botocore 1.27.59
    Uninstalling botocore-1.27.59:
      Successfully uninstalled botocore-1.27.59
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.4.2 requires botocore<1.27.60,>=1.27.59, but you have 

In [5]:
!pip install keras_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.8 MB/s eta 0:00:00


In [6]:
! pip install scikit-learn

In [7]:
! pip install transformers

In [8]:
import pandas as pd
import math
import numpy as np
from seqeval.metrics import f1_score
from seqeval.metrics import classification_report,accuracy_score,f1_score
import torch.nn.functional as F

from transformers import RobertaTokenizer,RobertaForTokenClassification
import torch
import os
from tqdm import tqdm,trange
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from pytorch_transformers import BertTokenizer, BertConfig
from pytorch_transformers import BertForTokenClassification, AdamW

In [9]:
df_train = pd.read_csv("/kaggle/input/jargon-detection/train.csv",sep=",",encoding="latin1")
df_train.head()

,token_id,token,label
0,cs/doc_1/0/0,A,O
1,cs/doc_1/0/1,SHAP,O
2,cs/doc_1/0/2,summary,O
3,cs/doc_1/0/3,plot,TERM
4,cs/doc_1/0/4,shows,O


In [10]:
df_val = pd.read_csv("/kaggle/input/jargon-detection/validation.csv",sep=",",encoding="latin1")
df_val.head()

,token_id,token,label
0,cs/doc_1867/0/0,Section,O
1,cs/doc_1867/0/1,~,O
2,cs/doc_1867/0/2,illustrates,O
3,cs/doc_1867/0/3,the,O
4,cs/doc_1867/0/4,proposed,O


In [11]:
df_test = pd.read_csv("/kaggle/input/jargon-detection/test_no_labels.csv",sep=",",encoding="latin1")
df_test.shape

(42358, 3)

In [12]:
df_val.label.unique()

array(['O', 'TERM'], dtype=object)

In [13]:
df_train[['m','n','x','y']] = df_train["token_id"].apply(lambda x: pd.Series(str(x).split("/")))

In [14]:
df_train.label.value_counts()

O       520286
TERM     54624
Name: label, dtype: int64

In [15]:
df_val.label.value_counts()

O       33479
TERM     3664
Name: label, dtype: int64

In [16]:
df_test.label.value_counts()

O    42358
Name: label, dtype: int64

In [17]:
df_val[['m','n','x','y']] = df_val["token_id"].apply(lambda x: pd.Series(str(x).split("/")))

In [18]:
df_test[['m','n','x','y']] = df_test["token_id"].apply(lambda x: pd.Series(str(x).split("/")))

In [19]:
df_train = df_train.fillna(' ') 

In [20]:
df_val = df_val.fillna(' ') 

In [21]:
df_test = df_test.fillna(' ') 

In [22]:
len(df_test)

42358

In [23]:
df_train[['a','b']] = df_train["n"].apply(lambda x: pd.Series(str(x).split("_")))
df_train

,token_id,token,label,m,n,x,y,a,b
0,cs/doc_1/0/0,A,O,cs,doc_1,0,0,doc,1
1,cs/doc_1/0/1,SHAP,O,cs,doc_1,0,1,doc,1
2,cs/doc_1/0/2,summary,O,cs,doc_1,0,2,doc,1
3,cs/doc_1/0/3,plot,TERM,cs,doc_1,0,3,doc,1
4,cs/doc_1/0/4,shows,O,cs,doc_1,0,4,doc,1
...,...,...,...,...,...,...,...,...,...
574905,phys/doc_1866/5/20,from,O,phys,doc_1866,5,20,doc,1866
574906,phys/doc_1866/5/21,a,O,phys,doc_1866,5,21,doc,1866
574907,phys/doc_1866/5/22,commercial,O,phys,doc_1866,5,22,doc,1866
574908,phys/doc_1866/5/23,supercapacitor,TERM,phys,doc_1866,5,23,doc,1866


In [24]:
df_val[['a','b']] = df_val["n"].apply(lambda x: pd.Series(str(x).split("_")))
df_val

,token_id,token,label,m,n,x,y,a,b
0,cs/doc_1867/0/0,Section,O,cs,doc_1867,0,0,doc,1867
1,cs/doc_1867/0/1,~,O,cs,doc_1867,0,1,doc,1867
2,cs/doc_1867/0/2,illustrates,O,cs,doc_1867,0,2,doc,1867
3,cs/doc_1867/0/3,the,O,cs,doc_1867,0,3,doc,1867
4,cs/doc_1867/0/4,proposed,O,cs,doc_1867,0,4,doc,1867
...,...,...,...,...,...,...,...,...,...
37138,phys/doc_1976/10/28,vorticity,TERM,phys,doc_1976,10,28,doc,1976
37139,phys/doc_1976/10/29,in,O,phys,doc_1976,10,29,doc,1976
37140,phys/doc_1976/10/30,the,O,phys,doc_1976,10,30,doc,1976
37141,phys/doc_1976/10/31,wake,O,phys,doc_1976,10,31,doc,1976


In [25]:
df_test[['a','b']] = df_test["n"].apply(lambda x: pd.Series(str(x).split("_")))
df_test

,token_id,token,label,m,n,x,y,a,b
0,cs/doc_1977/0/0,Next,O,cs,doc_1977,0,0,doc,1977
1,cs/doc_1977/0/1,",",O,cs,doc_1977,0,1,doc,1977
2,cs/doc_1977/0/2,we,O,cs,doc_1977,0,2,doc,1977
3,cs/doc_1977/0/3,analyze,O,cs,doc_1977,0,3,doc,1977
4,cs/doc_1977/0/4,the,O,cs,doc_1977,0,4,doc,1977
...,...,...,...,...,...,...,...,...,...
42353,phys/doc_2100/10/20,also,O,phys,doc_2100,10,20,doc,2100
42354,phys/doc_2100/10/21,presented,O,phys,doc_2100,10,21,doc,2100
42355,phys/doc_2100/10/22,and,O,phys,doc_2100,10,22,doc,2100
42356,phys/doc_2100/10/23,discussed,O,phys,doc_2100,10,23,doc,2100


In [26]:
df_train = df_train.drop('y', axis=1)
df_train = df_train.drop('token_id', axis=1)
df_train = df_train.drop('n', axis=1)
df_train = df_train.drop('a', axis=1)

In [27]:
df_val = df_val.drop('y', axis=1)
df_val = df_val.drop('token_id', axis=1)
df_val = df_val.drop('n', axis=1)
df_val = df_val.drop('a', axis=1)

In [28]:
df_test = df_test.drop('y', axis=1)
df_test = df_test.drop('token_id', axis=1)
df_test = df_test.drop('n', axis=1)
df_test = df_test.drop('a', axis=1)

In [29]:
df_train["s"] =  df_train["b"] + df_train['x'].astype(str) 

In [30]:
df_val["s"] = df_train["b"] + df_val['x'].astype(str)

In [31]:
df_test["s"] = df_test["b"] + df_test['x'].astype(str) 

In [32]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder() 

In [33]:
df_train['c']= label_encoder.fit_transform(df_train['m'])

In [34]:
#Encode labels in column 'species'.
df_val['c']= label_encoder.fit_transform(df_val['m'])
df_val['m'].unique()

array(['cs', 'econ', 'phys'], dtype=object)

In [35]:
# Encode labels in column 'species'.
df_test['c']= label_encoder.fit_transform(df_test['m'])
df_test['m'].unique()

array(['cs', 'econ', 'phys'], dtype=object)

In [36]:
df_test["s"] = df_test['c'].astype(str) + df_test["s"]

In [37]:
df_val["s"] = df_val['c'].astype(str) + df_train["s"]

In [38]:
df_train["s"] = df_train["c"].astype(str) + df_train["s"]

In [39]:
df_train = df_train.drop('x', axis=1)
df_train = df_train.drop('b', axis=1)
df_train = df_train.drop('c', axis=1)

In [40]:
df_val = df_val.drop('x', axis=1)
df_val = df_val.drop('b', axis=1)
df_val = df_val.drop('c', axis=1)

In [41]:
df_test = df_test.drop('x', axis=1)
df_test = df_test.drop('b', axis=1)
df_test = df_test.drop('c', axis=1)

In [42]:
class divider(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(t,m,l) for t,m,l in zip(s["token"].values.tolist(),
                                                           s["m"].values.tolist(),
                                                           s["label"].values.tolist())]
        self.grouped = self.data.groupby("s").apply(agg_func)
        self.sentences = [s for s in self.grouped]

In [43]:
getter_trn = divider(df_train)
sentences_trn = [[s[0] for s in sent] for sent in getter_trn.sentences]

In [44]:
getter_val = divider(df_val)
sentences_val = [[s[0] for s in sent] for sent in getter_val.sentences]

In [45]:
getter_tst = divider(df_test)
sentences_tst = [[str(s[0]) for s in sent] for sent in getter_tst.sentences]

In [46]:
topic_trn = [[s[1] for s in sent] for sent in getter_trn.sentences]
labels_trn = [[s[2] for s in sent] for sent in getter_trn.sentences]

In [47]:
topic_val = [[s[1] for s in sent] for sent in getter_val.sentences]
labels_val = [[s[2] for s in sent] for sent in getter_val.sentences]

In [48]:
topic_tst = [[s[1] for s in sent] for sent in getter_tst.sentences]
labels_tst = [[s[2] for s in sent] for sent in getter_tst.sentences]

In [49]:
tags_vals = list(set(df_train["label"].values))

In [50]:
tags_vals = set(tags_vals)
tags_vals

{'O', 'TERM'}

In [51]:
tagtoid={'O': 0,
         'TERM' : 1,
         }

In [52]:
idtotag={tagtoid[key] : key for key in tagtoid.keys()}

In [53]:
device = torch.device("cuda" )if torch.cuda.is_available() else "cpu"
n_gpu = torch.cuda.device_count()

In [54]:
device

device(type='cuda')

In [55]:
# Len of the sentence must be the same as the training model
# See model's 'max_position_embeddings' = 512
max_len  = 40
batch_num = 64
# load tokenizer, with manual file address or pretrained address

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [56]:
sen_tst = []
for i in sentences_tst :
    for j in i:
        sen_tst.append(str(j))

In [57]:
sen_trn = []
for i in sentences_trn :
    for j in i:
        sen_trn.append(str(j))
lab_trn = []
for i in labels_trn :
    for j in i:
        lab_trn.append(str(j))   

In [58]:
sen_val = []
for i in sentences_val :
    for j in i:
        sen_val.append(str(j))
lab_val = []
for i in labels_val :
    for j in i:
        lab_val.append(str(j))        
        

In [59]:
tokenized_texts_tst = [sen_tst[i:i+40] for i in range(0, len(sen_tst), 40)]
tokenized_texts_tst ;   

In [60]:
tokenized_texts_trn = [sen_trn[i:i+40] for i in range(0, len(sen_trn), 40)]
tokenized_texts_trn ;   
word_piece_labels_trn = [lab_trn[i:i+40] for i in range(0, len(lab_trn), 40)]
word_piece_labels_trn ;  

In [61]:
tokenized_texts_val = [sen_val[i:i+40] for i in range(0, len(sen_val), 40)]
tokenized_texts_val ;   
word_piece_labels_val = [lab_val[i:i+40] for i in range(0, len(lab_val), 40)]
word_piece_labels_val ;   

In [62]:
input_ids_trn = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_trn],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids_trn[0])

[  250     3 48600 40776 35624   627     3 19434  1116 46076  1990 42333
 23687 21996     4 13863 29902 43457  1322 44661     3     3 29888     3
     6 30117 10010  1322  5525 24235   560     3    73     3   179   627
 32530  8738 34437     6]


In [63]:
input_ids_val = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_val],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids_val[0])

[43480 34437     3   627     3     3 39165     3 45416     4 43480 34437
 35624   627 19675     6   627 44240 47693 10799 17304  1610 28431     3
  5652     3     3     6   463 32931   405  7333   102     3 47438 10799
     4  1121  9226 33331]


In [64]:
input_ids_tst = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_tst],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids_tst[0])

[19192     6  1694     3   627 48360 36274 42342  1640     3 40683    43
     4   250 48360 36274 42342   354   627 36274  1116   102     3 48360
  5488   354  4321  5652   102     3 19434  1640     3     6  2940     3
  1116   627 41976 30695]


In [65]:
input_ids_tst[15]

array([  560,     3,   179,    12, 48024,     3,     4,   170,     3,
        6025,  2362,     3,  9764,     3,     3,  1990,   627,   179,
          12, 48024,     3,     3,     4,   170, 34996,   627,     3,
        1116,   627, 30695,  1116, 14616,     3, 34980,  1640,     3,
         463, 41976,     3, 14035])

In [66]:
tags_trn = pad_sequences([[tagtoid.get(l) for l in lab] for lab in word_piece_labels_trn],
                     maxlen=max_len, value=tagtoid["O"], padding="post",
                     dtype="long", truncating="post")
print(tags_trn[0])

[0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0]


In [67]:
tags_val = pad_sequences([[tagtoid.get(l) for l in lab] for lab in word_piece_labels_val],
                     maxlen=max_len, value=tagtoid["O"], padding="post",
                     dtype="long", truncating="post")
print(tags_val[0])

[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0]


In [68]:
# For fine tune of predict, with token mask is 1,pad token is 0
attention_masks_trn = [[int(i>0) for i in ii] for ii in input_ids_trn]
attention_masks_trn[0];

In [69]:
# For fine tune of predict, with token mask is 1,pad token is 0
attention_masks_val = [[int(i>0) for i in ii] for ii in input_ids_val]
attention_masks_val[0];

In [70]:
# For fine tune of predict, with token mask is 1,pad token is 0
attention_masks_tst = [[int(i>0) for i in ii] for ii in input_ids_tst]
attention_masks_tst[0];

In [71]:
# Since only one sentence, all the segment set to 0
# segment_ids_trn = [[0] * len(input_id) for input_id in input_ids_trn]
# segment_ids_trn[0];

In [72]:
# # Since only one sentence, all the segment set to 0
# segment_ids_val = [[0] * len(input_id) for input_id in input_ids_val]
# segment_ids_val[0];

In [73]:
# # Since only one sentence, all the segment set to 0
# segment_ids_tst = [[0] * len(input_id) for input_id in input_ids_tst]
# segment_ids_tst[0];

In [74]:
tr_inputs = torch.tensor(input_ids_trn)
val_inputs = torch.tensor(input_ids_val)
tst_inputs = torch.tensor(input_ids_tst)

tr_tags = torch.tensor(tags_trn)
val_tags = torch.tensor(tags_val)

tr_masks = torch.tensor(attention_masks_trn)
val_masks = torch.tensor(attention_masks_val)
tst_masks = torch.tensor(attention_masks_tst)

In [75]:
# Only set token embedding, attention embedding, no segment embedding
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_num,drop_last=True)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_num)

In [76]:
test_data = TensorDataset(tst_inputs, tst_masks)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_num)

In [77]:
model = RobertaForTokenClassification.from_pretrained('roberta-base',num_labels=len(tagtoid))

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able

In [78]:
model = model.to(device)

In [79]:
# Set epoch and grad max num
epochs = 15
max_grad_norm = 1.0

In [80]:
# Cacluate train optimiazaion num
num_train_optimization_steps = int( math.ceil(len(tr_inputs) / batch_num) / 1) * epochs

In [81]:
# Fine tune model all layer parameters
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
    ]

In [82]:
# TRAIN loop
model.train();

In [83]:
optimizer = AdamW(optimizer_grouped_parameters, lr=0.0001)

## Training phase

In [84]:
for _ in trange(epochs,desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        # forward pass
        outputs = model(b_input_ids, token_type_ids=None,
        attention_mask=b_input_mask, labels=b_labels)
        loss, scores = outputs[:2]
        if n_gpu>1:
            # When multi gpu, average it
            loss = loss.mean()
        
        # backward pass
        loss.backward()
        
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        
        # update parameters
        optimizer.step()
        optimizer.zero_grad()
        
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/pytorch_transformers/optimization.py:166: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/python_arg_parser.cpp:1420.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)
Epoch:   7%|▋         | 1/15 [01:01<14:15, 61.10s/it]

Train loss: 0.1772022470831871


Epoch:  13%|█▎        | 2/15 [02:00<13:03, 60.24s/it]

Train loss: 0.14399687500138367


Epoch:  20%|██        | 3/15 [03:00<11:59, 59.97s/it]

...

Train loss: 0.03185393649619073


Epoch:  93%|█████████▎| 14/15 [13:58<00:59, 59.84s/it]

Train loss: 0.02823693660320714


Epoch: 100%|██████████| 15/15 [14:58<00:00, 59.87s/it]

Train loss: 0.024312543200461993


In [85]:
model_saving_address = 'models/final'
# Make dir if not exits
if not os.path.exists(model_saving_address):
    os.makedirs(model_saving_address)

In [86]:
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self

In [87]:
# If we save using the predefined names, we can load using `from_pretrained`
model_file_location = os.path.join(model_saving_address, "pytorch_model.bin")
configuration_file_location = os.path.join(model_saving_address, "config.json")

# Save model into file
torch.save(model_to_save.state_dict(), model_file_location)
model_to_save.config.to_json_file(configuration_file_location)
tokenizer.save_vocabulary(model_saving_address)

('models/final/vocab.json', 'models/final/merges.txt')

In [88]:
model.eval();

## Evaluation phase

In [89]:
y_true = []
y_pred = []

for step, batch in enumerate(valid_dataloader):
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, label = batch
    
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None,attention_mask=input_mask,)
        logits = outputs[0] 
   
    logits = torch.argmax(F.log_softmax(logits,dim=2),dim=2)
    logits = logits.detach().cpu().numpy()
    
    label = label.to('cpu').numpy()
    
    input_mask = input_mask.to('cpu').numpy()
    
    for i,mask in enumerate(input_mask):
       # creating two empty lists.
        lst_1 = []
        lst_2 = []
        
        for j, m in enumerate(mask):
            if m:
                lst_1.append(idtotag[label[i][j]])
                lst_2.append(idtotag[logits[i][j]])
            else:
                break
        
            
        y_true.append(lst_1)
        y_pred.append(lst_2)
# the report
report = classification_report(y_true, y_pred,digits=4)


print("\n%s"%(report))
# F1 score
print("Final F1 score: %f"%(f1_score(y_true, y_pred)))
# Accuracy Score
print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))



           precision    recall  f1-score   support

     TERM     0.6484    0.6195    0.6337      3656

micro avg     0.6484    0.6195    0.6337      3656
macro avg     0.6484    0.6195    0.6337      3656

Final F1 score: 0.633655
Accuracy score: 0.929677


## Training phase

In [90]:
# Model we trained before, the dir containing pytorch_model.bin and vocab.txt
save_model_address = 'models/final'

In [91]:
save_model = BertForTokenClassification.from_pretrained(save_model_address,num_labels=len(tagtoid))

In [92]:
# Get model predict result
with torch.no_grad():
        outputs = save_model(tst_inputs, token_type_ids=None,attention_mask=None,)
        # For eval mode, the first result of outputs is logits
        #logits = outputs[0] 

In [93]:
# Make logits into numpy type predict result
# The predict result contain each token's all tags predict result
predict_results = outputs[0].detach().cpu().numpy()

In [94]:
predict_results.shape

(1059, 40, 2)

In [95]:
from scipy.special import softmax

In [96]:
result_list = []
for pre_res in predict_results:
    result_arrays_soft =[]
    for i in pre_res:
        result_arrays_soft.append(softmax(i))
    result_list.append(np.argmax(result_arrays_soft,axis= -1))    

In [97]:
print(result_list[9])

[1 0 1 0 0 0 1 1 1 1 1 1 0 0 0 1 0 1 1 1 0 1 1 1 0 0 0 0 0 0 1 0 1 1 1 1 1
 0 1 0]


In [98]:
print(input_ids_tst[15])

[  560     3   179    12 48024     3     4   170     3  6025  2362     3
  9764     3     3  1990   627   179    12 48024     3     3     4   170
 34996   627     3  1116   627 30695  1116 14616     3 34980  1640     3
   463 41976     3 14035]


In [99]:
print(tst_masks[9])

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


In [100]:
# answer = []
# for j in y_pred: 
#     for i in j:  
#         answer.append(i)      

In [101]:
answer = []
for j in range(len(result_list)): 
    for i, mark in enumerate(tst_masks[j]):
        if mark>0:
            if result_list[j][i] == 1 :   
                answer.append('TERM')      
            else :   
                answer.append('O')

In [102]:
len(answer)

42358

In [103]:
submission_df = pd.read_csv("/kaggle/input/jargon-detection/test_no_labels.csv",sep=",",encoding="latin1")

In [104]:
submission_df['label'] = pd.DataFrame(answer)

In [105]:
submission_df = submission_df.drop(['token'], axis=1)

In [106]:
submission_df.to_csv("Roberta_base.csv", index=False)